In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

# set the aesthetic style of the plots
sns.set_style()

# filter warning messages
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


In [2]:
data_dict = pd.read_excel('data/Data Dictionary.xls')

In [3]:
data_dict

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Variable Name,Description,Type
1,SeriousDlqin2yrs,Person experienced 90 days past due delinquenc...,Y/N
2,RevolvingUtilizationOfUnsecuredLines,Total balance on credit cards and personal lin...,percentage
3,age,Age of borrower in years,integer
4,NumberOfTime30-59DaysPastDueNotWorse,Number of times borrower has been 30-59 days p...,integer
5,DebtRatio,"Monthly debt payments, alimony,living costs di...",percentage
6,MonthlyIncome,Monthly income,real
7,NumberOfOpenCreditLinesAndLoans,Number of Open loans (installment like car loa...,integer
8,NumberOfTimes90DaysLate,Number of times borrower has been 90 days or m...,integer
9,NumberRealEstateLoansOrLines,Number of mortgage and real estate loans inclu...,integer


In [4]:
# read data
df_train = pd.read_csv('data/cs-training.csv').iloc[:, 1:]
df_train['set'] = 'train'

In [5]:
# read data
df_test = pd.read_csv('data/cs-test.csv').iloc[:, 1:]
df_test['set'] = 'test'

In [6]:
df = pd.concat([df_train, df_test], axis = 0)

In [7]:
df['index'] = range(0, len(df))

# Descriptive Analysis 

In [8]:
print('Number of rows: ', df.shape[0])
print('Number of columns: ', df.shape[1])

Number of rows:  251503
Number of columns:  13


There are 11 features for 251,503 clients/users. The target variable we are predicting for is 'SeriousDlqin2yrs', which contains a count of delinquency events over a period of time. We will explore all features to treat possible missing values and make oher necessary adjustments to improve the overall quality of the model.

In [12]:
# dataframe summary
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251503 entries, 0 to 101502
Data columns (total 13 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  float64
 1   RevolvingUtilizationOfUnsecuredLines  251503 non-null  float64
 2   age                                   251503 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  251503 non-null  int64  
 4   DebtRatio                             251503 non-null  float64
 5   MonthlyIncome                         201669 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       251503 non-null  int64  
 7   NumberOfTimes90DaysLate               251503 non-null  int64  
 8   NumberRealEstateLoansOrLines          251503 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  251503 non-null  int64  
 10  NumberOfDependents                    244953 non-null  float64
 11  

There are originally 11 numerical columns. I note that the features seem to be quite engineered already, and don't exist (besides 'age') as the raw-level data. 

In [13]:
df_test.isna().sum()

SeriousDlqin2yrs                        101503
RevolvingUtilizationOfUnsecuredLines         0
age                                          0
NumberOfTime30-59DaysPastDueNotWorse         0
DebtRatio                                    0
MonthlyIncome                            20103
NumberOfOpenCreditLinesAndLoans              0
NumberOfTimes90DaysLate                      0
NumberRealEstateLoansOrLines                 0
NumberOfTime60-89DaysPastDueNotWorse         0
NumberOfDependents                        2626
set                                          0
dtype: int64

In [14]:
# percentage of missing values per feature
print((df.isnull().sum() * 100 / df.shape[0]).sort_values(ascending=False))

SeriousDlqin2yrs                        40.358564
MonthlyIncome                           19.814475
NumberOfDependents                       2.604343
index                                    0.000000
set                                      0.000000
NumberOfTime60-89DaysPastDueNotWorse     0.000000
NumberRealEstateLoansOrLines             0.000000
NumberOfTimes90DaysLate                  0.000000
NumberOfOpenCreditLinesAndLoans          0.000000
DebtRatio                                0.000000
NumberOfTime30-59DaysPastDueNotWorse     0.000000
age                                      0.000000
RevolvingUtilizationOfUnsecuredLines     0.000000
dtype: float64


We note several missing values. The most critical case is 'MonthlyIncome', which has 19.8% (20k) missing values. This presents a problem because intuitively, income is an important indicator of the borrower's ability to repay (later confirmed by plotting the feature importance plot). 

In [15]:
# number of unique observations per column
df.nunique().sort_values()

SeriousDlqin2yrs                             2
set                                          2
NumberOfTime60-89DaysPastDueNotWorse        13
NumberOfDependents                          14
NumberOfTime30-59DaysPastDueNotWorse        17
NumberOfTimes90DaysLate                     21
NumberRealEstateLoansOrLines                29
NumberOfOpenCreditLinesAndLoans             60
age                                         88
MonthlyIncome                            15757
DebtRatio                               182595
RevolvingUtilizationOfUnsecuredLines    208075
index                                   251503
dtype: int64

In [16]:
df.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,index
count,150000.000000,251503.000000,251503.000000,251503.000000,251503.000000,2.016690e+05,251503.000000,251503.000000,251503.000000,251503.000000,244953.000000,251503.000000
mean,0.066840,5.750415,52.339694,0.434245,349.562468,6.744818e+03,8.453064,0.278370,1.016155,0.252466,0.761995,125751.000000
std,0.249746,229.633980,14.775120,4.335643,1884.792016,2.571761e+04,5.145194,4.312539,1.121935,4.299204,1.123905,72602.806713
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029977,41.000000,0.000000,0.174330,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000,62875.500000
50%,0.000000,0.153575,52.000000,0.000000,0.365612,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000,125751.000000
75%,0.000000,0.561293,63.000000,0.000000,0.861754,8.212000e+03,11.000000,0.000000,2.000000,0.000000,1.000000,188626.500000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,7.727000e+06,85.000000,98.000000,54.000000,98.000000,43.000000,251502.000000


# Correlation Analysis

In [17]:
import pandas as pd
import numpy as np

corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,index
SeriousDlqin2yrs,1.000000,-0.001802,-0.115386,0.125587,-0.007602,-0.019746,-0.029669,0.117175,-0.007038,0.102261,0.046048,0.002801
RevolvingUtilizationOfUnsecuredLines,-0.001802,1.000000,-0.006324,-0.001040,0.003929,0.002969,-0.011148,-0.001147,0.005433,-0.000993,0.001675,-0.000327
age,-0.115386,-0.006324,1.000000,-0.065567,0.027648,0.025466,0.146123,-0.063735,0.030566,-0.059642,-0.214841,0.004982
NumberOfTime30-59DaysPastDueNotWorse,0.125587,-0.001040,-0.065567,1.000000,-0.007220,-0.006368,-0.057645,0.984524,-0.032571,0.987799,-0.002360,0.002271
DebtRatio,-0.007602,0.003929,0.027648,-0.007220,1.000000,-0.010675,0.054723,-0.009265,0.122663,-0.008354,-0.042900,-0.003044
MonthlyIncome,-0.019746,0.002969,0.025466,-0.006368,-0.010675,1.000000,0.050622,-0.007546,0.069845,-0.006662,0.032861,0.002667
NumberOfOpenCreditLinesAndLoans,-0.029669,-0.011148,0.146123,-0.057645,0.054723,0.050622,1.000000,-0.082029,0.432817,-0.073180,0.065066,0.002445
NumberOfTimes90DaysLate,0.117175,-0.001147,-0.063735,0.984524,-0.009265,-0.007546,-0.082029,1.000000,-0.046819,0.993378,-0.009859,0.001932
NumberRealEstateLoansOrLines,-0.007038,0.005433,0.030566,-0.032571,0.122663,0.069845,0.432817,-0.046819,1.000000,-0.041177,0.123216,-0.001909
NumberOfTime60-89DaysPastDueNotWorse,0.102261,-0.000993,-0.059642,0.987799,-0.008354,-0.006662,-0.073180,0.993378,-0.041177,1.000000,-0.010706,0.001957


In [65]:
def get_redundant_pairs(df):
    """
    Retrieve redundant bivariate pairs

    # Arguments
        df: DataFrame containing the independent variables.
        
    # Returns
        pairs_to_drop: pairs of variables
    """
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_correlations(df, n=5):
    """
    Retrieve top correlations

    # Arguments
        df: DataFrame containing the independent variables
        
    # Returns
        au_corr: Table of bivariate pairs and corresponding variables
    """
    
    au_corr = df.corr().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending= False)
    return au_corr[0:n]

print("Top Correlations")
print(get_top_correlations(df.drop(columns = 'set'), 50))

Top Correlations
NumberOfTimes90DaysLate               NumberOfTime60-89DaysPastDueNotWorse    0.993378
NumberOfTime30-59DaysPastDueNotWorse  NumberOfTime60-89DaysPastDueNotWorse    0.987799
                                      NumberOfTimes90DaysLate                 0.984524
NumberOfOpenCreditLinesAndLoans       NumberRealEstateLoansOrLines            0.432817
age                                   NumberOfOpenCreditLinesAndLoans         0.146123
SeriousDlqin2yrs                      NumberOfTime30-59DaysPastDueNotWorse    0.125587
NumberRealEstateLoansOrLines          NumberOfDependents                      0.123216
DebtRatio                             NumberRealEstateLoansOrLines            0.122663
SeriousDlqin2yrs                      NumberOfTimes90DaysLate                 0.117175
                                      NumberOfTime60-89DaysPastDueNotWorse    0.102261
MonthlyIncome                         NumberRealEstateLoansOrLines            0.069845
NumberOfOpenCreditLinesAnd

In [66]:
corr_df = pd.DataFrame(get_top_correlations(df2, 300)).reset_index()
corr_df.columns = ['var_1', 'var_2', 'correlation']

In [68]:
corr_df

,var_1,var_2,correlation
0,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,0.993378
1,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,0.987799
2,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTimes90DaysLate,0.984524
3,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,0.432817
4,age,NumberOfOpenCreditLinesAndLoans,0.146123
5,NumberRealEstateLoansOrLines,NumberOfDependents,0.123216
6,DebtRatio,NumberRealEstateLoansOrLines,0.122663
7,MonthlyIncome,NumberRealEstateLoansOrLines,0.069845
8,NumberOfOpenCreditLinesAndLoans,NumberOfDependents,0.065066
9,DebtRatio,NumberOfOpenCreditLinesAndLoans,0.054723


Looking at the correlations give us some observations and ideas for feature engineering

*Observations*
Most correlations are not surprising and intuitive. In summary there are strong positive correlations dueness and lateness, income and loans, loans and dependents.

*Ideas for Feature Engineering*
Given the above relationships, there is room for more features to be engineered to create more indicators of users having overdrawn or overleveraged. I will later engineer the following

1) ratio of due instances to number of loans
2) ratio of late instances to number of loans
3) median and average aggregates by age
4) median and average aggregates by income

# Treating Missing Values
It's time to deal with the missing values according to the particularities of each feature. 

I use iterative_imputer, which is more accurate than simply filling with mean, median, or mode values. Iterative Imputer will predict each missing feature as a function of all other features and to repeat this process of estimating feature values multiple times. The repetition allows the refined estimated values for other features to be used as input in subsequent iterations of predicting missing values. Essentially, missing values get filled according to what other features predict, which is more likely to be accurate than filling with central tendency values. 

In [48]:
df2 = df.drop(columns = ['set','SeriousDlqin2yrs'])

In [49]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
imp = IterativeImputer(estimator = lr, verbose =2, max_iter = 30, tol = 1e-10, imputation_order = 'roman', min_value =0  )

In [50]:
imp.fit(df2)

[IterativeImputer] Completing matrix with shape (251503, 11)
[IterativeImputer] Ending imputation round 1/30, elapsed time 1.79
[IterativeImputer] Change: 19523.162592785695, scaled tolerance: 0.0007727000000000001 
[IterativeImputer] Ending imputation round 2/30, elapsed time 3.64
[IterativeImputer] Change: 909.3866945806425, scaled tolerance: 0.0007727000000000001 
[IterativeImputer] Ending imputation round 3/30, elapsed time 5.54
[IterativeImputer] Change: 0.946319662513019, scaled tolerance: 0.0007727000000000001 
[IterativeImputer] Ending imputation round 4/30, elapsed time 7.45
[IterativeImputer] Change: 0.0009847525910613442, scaled tolerance: 0.0007727000000000001 
[IterativeImputer] Ending imputation round 5/30, elapsed time 9.32
[IterativeImputer] Change: 1.0247474340197016e-06, scaled tolerance: 0.0007727000000000001 
[IterativeImputer] Early stopping criterion reached.


IterativeImputer(estimator=LinearRegression(), imputation_order='roman',
                 max_iter=30, min_value=0, tol=1e-10, verbose=2)

In [51]:
df3 = imp.transform(df2)

[IterativeImputer] Completing matrix with shape (251503, 11)
[IterativeImputer] Ending imputation round 1/5, elapsed time 0.03
[IterativeImputer] Ending imputation round 2/5, elapsed time 0.08
[IterativeImputer] Ending imputation round 3/5, elapsed time 0.10
[IterativeImputer] Ending imputation round 4/5, elapsed time 0.16
[IterativeImputer] Ending imputation round 5/5, elapsed time 0.18


In [52]:
df4 = pd.DataFrame(df3, columns = df2.columns)

In [53]:
import math
df4['NumberofDependents_rounded'] = df4['NumberOfDependents'].apply(lambda x: math.floor(x))

In [56]:
addn = df[['index', 'set', 'SeriousDlqin2yrs']]

In [60]:
# after cleaning, I reconstitute the dataset to pass to the next phase
df6 = df4.merge(addn, on = 'index')

In [61]:
df6

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,index,NumberofDependents_rounded,set,SeriousDlqin2yrs
0,0.766127,45.0,2.0,0.802982,9120.000000,13.0,0.0,6.0,0.0,2.000000,0.0,2,train,1.0
1,0.957151,40.0,0.0,0.121876,2600.000000,4.0,0.0,0.0,0.0,1.000000,1.0,1,train,0.0
2,0.658180,38.0,1.0,0.085113,3042.000000,2.0,1.0,0.0,0.0,0.000000,2.0,0,train,0.0
3,0.233810,30.0,0.0,0.036050,3300.000000,5.0,0.0,0.0,0.0,0.000000,3.0,0,train,0.0
4,0.907239,49.0,1.0,0.024926,63588.000000,7.0,0.0,1.0,0.0,0.000000,4.0,0,train,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251498,0.282653,24.0,0.0,0.068522,1400.000000,5.0,0.0,0.0,0.0,0.000000,251498.0,0,test,NaN
251499,0.922156,36.0,3.0,0.934217,7615.000000,8.0,0.0,2.0,0.0,4.000000,251499.0,4,test,NaN
251500,0.081596,70.0,0.0,836.000000,4990.493343,3.0,0.0,0.0,0.0,0.261426,251500.0,0,test,NaN
251501,0.335457,56.0,0.0,3568.000000,8265.053320,8.0,0.0,2.0,1.0,3.000000,251501.0,3,test,NaN


In [62]:
df6.to_csv('processed_data/df.csv')

After handling the missing values, case by case, we now have a train data set free of null values, outside of the intentionally deleted test set values. 